In [ ]:
!pip install transformers -q
!pip install wandb -q
!pip install sentencepiece

     |████████████████████████████████| 1.4MB 17.2MB/s 
     |████████████████████████████████| 890kB 57.1MB/s 
     |████████████████████████████████| 2.9MB 54.0MB/s 
     |████████████████████████████████| 1.8MB 15.3MB/s 
     |████████████████████████████████| 163kB 58.9MB/s 
     |████████████████████████████████| 102kB 12.6MB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 133kB 51.7MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 1.1MB 15.0MB/s 


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

# WandB – Import the wandb library
import wandb

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
model =BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
model = model.to(device)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
test_path="./new_test.csv"

Mounted at /content/drive


In [ ]:
#load the model, remember to change it to the actual model name you trained
model.load_state_dict(torch.load('./BART_epoch5.pth'))

<All keys matched successfully>

In [ ]:
#testing method
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
#data loader
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.subject
        self.ctext = self.data.content

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())
        source = self.tokenizer.batch_encode_plus([ctext], padding='max_length', truncation=True,max_length= self.source_len,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text],  padding='max_length', truncation=True,max_length= self.summ_len,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long)
        }

In [ ]:
df_test=pd.read_csv(test_path,encoding='latin-1')
df_test=df_test.dropna()
df_test=df_test.reset_index(drop=True)

In [ ]:
val_params = {
    'batch_size': 2,
    'shuffle': False,
    'num_workers': 0
    }

In [ ]:
#512 is for the max_length of input, 15 is the max_len for the summary
val_set = CustomDataset(df_test, tokenizer, 512, 15)
val_loader = DataLoader(val_set, **val_params)

In [ ]:
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(1):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    #remember to change the file name to the name you want
    final_df.to_csv('./BART_5epoch_basic.csv')
    print('Output Files generated for review')

Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Output Files generated for review
